In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

hamzahabib47_brain_cancer_detection_mri_images_path = kagglehub.dataset_download('hamzahabib47/brain-cancer-detection-mri-images')
abhyudayvenkatesh_braincancer_path = kagglehub.dataset_download('abhyudayvenkatesh/braincancer')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPooling2D
import os
from sklearn.metrics import confusion_matrix,classification_report

# Preprocessing

In [ ]:
# Set up the path for your dataset
dataset_path = "/kaggle/input/brain-cancer-detection-mri-images/Dataset"
healthy_path = os.path.join(dataset_path, 'healthy')
tumor_path = os.path.join(dataset_path, 'tumor')

# Constants
img_size = 128
input_shape = (img_size, img_size, 3)

# Helper functions for loading and preprocessing the dataset
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = tf.keras.preprocessing.image.load_img(os.path.join(folder, filename), target_size=(img_size, img_size))
        if img is not None:
            images.append(tf.keras.preprocessing.image.img_to_array(img))
    return images

# Load images for healthy and tumor categories
healthy_images = load_images_from_folder(healthy_path)
tumor_images = load_images_from_folder(tumor_path)

# Labels for the dataset
healthy_labels = [0] * len(healthy_images)
tumor_labels = [1] * len(tumor_images)

# Combine and reshape
X = np.array(healthy_images + tumor_images)
y = np.array(healthy_labels + tumor_labels)

# Visualization

In [ ]:
# Visualize a few sample images from the dataset, showing both healthy and tumor images
def show_sample_images(X, y, num_samples=10):
    plt.figure(figsize=(15, 10))
    healthy_count = 0
    tumor_count = 0
    for i in range(num_samples):
        plt.subplot(2, num_samples // 2, i + 1)
        if healthy_count < num_samples // 2:
            plt.imshow(X[healthy_count])
            plt.title('Healthy', fontsize=12)
            healthy_count += 1
        else:
            plt.imshow(X[len(healthy_images) + tumor_count])
            plt.title('Tumor', fontsize=12)
            tumor_count += 1
        plt.axis('off')
    plt.show()

# Show sample images of both healthy and tumor
show_sample_images(X, y)

In [ ]:
sns.countplot(x=y)
plt.title('Distribution of Healthy and Tumor Images', fontsize=14)
plt.xticks([0, 1], ['Healthy', 'Tumor'], fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Label', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.show()

# Data Augmentation

In [ ]:
# Normalize the images
X = X / 255.0
X, y = shuffle(X, y, random_state=42)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
 cccvb   rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=16,shuffle=True)

# Model Preperation and Training

# Using ResNet with normal and Augmented Data

In [ ]:
input_shape = (128, 128, 3)
metrics={}

In [ ]:
resnet_model=Sequential()

pretrained_model=tf.keras.applications.ResNet50(include_top=False,
                                              input_shape=input_shape,
                                              pooling='max',classes=2,
                                              weights='imagenet')
for layer in pretrained_model.layers:
    layers.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(layers.Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(layers.Dropout(0.5))
resnet_model.add(Dense(1,activation='sigmoid'))
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
loss='binary_crossentropy',
metrics=['accuracy'])

In [ ]:
resnet_model_aug=Sequential()

pretrained_model=tf.keras.applications.ResNet50(include_top=False,
                                              input_shape=input_shape,
                                              pooling='max',classes=2,
                                              weights='imagenet')
for layer in pretrained_model.layers:
    layers.trainable=False

resnet_model_aug.add(pretrained_model)
resnet_model_aug.add(layers.Flatten())
resnet_model_aug.add(Dense(512, activation='relu'))
resnet_model_aug.add(layers.Dropout(0.5))
resnet_model_aug.add(Dense(1,activation='sigmoid'))
resnet_model_aug.summary()

In [ ]:
resnet_model_aug.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
loss='binary_crossentropy',
metrics=['accuracy'])

In [ ]:
metrics['resnet_history']=resnet_model.fit(X_train,y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
metrics['resnet_aug_history'] = resnet_model_aug.fit(train_generator, epochs=10, validation_data=(X_test, y_test))

# Using Conv2D using normal and Augmented Data

In [ ]:
cnn_model=Sequential()
cnn_model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(layers.Flatten())
cnn_model.add(Dense(128,activation='relu'))
cnn_model.add(Dense(1,activation='sigmoid'))
cnn_model.summary()

In [ ]:
 cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
cnn_model_aug=Sequential()
cnn_model_aug.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
cnn_model_aug.add(MaxPooling2D(pool_size=(2,2)))
cnn_model_aug.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
cnn_model_aug.add(MaxPooling2D(pool_size=(2,2)))
cnn_model_aug.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
cnn_model_aug.add(MaxPooling2D(pool_size=(2,2)))
cnn_model_aug.add(layers.Flatten())
cnn_model_aug.add(Dense(128,activation='relu'))
cnn_model_aug.add(Dense(1,activation='sigmoid'))
cnn_model_aug.summary()

In [ ]:
 cnn_model_aug.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
metrics['cnn_history'] = cnn_model.fit(X_train,y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
metrics['cnn_aug_history'] = cnn_model_aug.fit(train_generator, epochs=10, validation_data=(X_test, y_test))

# Testing and Evaluation and Visualization

In [ ]:

metrics['resnet_y_pred'] = (resnet_model.predict(X_test)>0.5).astype(int)
metrics['resnet_aug_y_pred']=(resnet_model_aug.predict(X_test)>0.5).astype(int)
metrics['cnn_y_pred']=(cnn_model.predict(X_test)>0.5).astype(int)
metrics['cnn_aug_y_pred']=(cnn_model_aug.predict(X_test)>0.5).astype(int)

In [ ]:
metrics

In [ ]:
models=['cnn','resnet','cnn_aug','resnet_aug']
all_histories=[]
for i in models:
    all_histories.append(metrics[f'{i}_history'])

for i,name in enumerate(models):
    plt.figsize=(10,10)
    plt.plot(all_histories[i].history['accuracy'],label='accuracy')
    plt.plot(all_histories[i].history['val_accuracy'],label='val_accuracy')
    plt.title(f'{name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.show()
     # Print classification report
    print(f"Classification report for {name}:")
    print(classification_report(y_test,metrics[f'{name}_y_pred']))

    # Confusion Matrix
    cm = confusion_matrix(y_test,metrics[f'{name}_y_pred'])

    # Confusion Matrix Visualization
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='mako', cbar=True,
                xticklabels=['Healthy', 'Tumor'], yticklabels=['Healthy', 'Tumor'])
    plt.title(f'Confusion Matrix for {name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

# Aggregate Model Summary

In [ ]:
metrics_summary=[]

for i,name in enumerate(models):

    final_train_accuracy=metrics[f'{name}_history'].history['accuracy'][-1]
    final_val_accuracy=metrics[f'{name}_history'].history['val_accuracy'][-1]
    final_train_loss=metrics[f'{name}_history'].history['loss'][-1]
    final_val_loss=metrics[f'{name}_history'].history['val_loss'][-1]
    metrics_summary.append({
            'Model': name,
            'Final Train Accuracy': final_train_accuracy,
            'Final Validation Accuracy': final_val_accuracy,
            'Final Train Loss': final_train_loss,
            'Final Validation Loss': final_val_loss
        })
metrics_df=pd.DataFrame(metrics_summary)
metrics_df

In [ ]:
# Plotting Accuracy Comparison
plt.figure(figsize=(12, 6))
plt.bar(metrics_df['Model'], metrics_df['Final Validation Accuracy'], color='skyblue')
plt.title('Final Validation Accuracy Comparison')
plt.xlabel('Model')
plt.ylabel('Validation Accuracy')
plt.xticks(rotation=45)
plt.ylim([0, 1])
plt.grid(axis='y')
plt.show()

# Plotting Loss Comparison
plt.figure(figsize=(12, 6))
plt.bar(metrics_df['Model'], metrics_df['Final Validation Loss'], color='salmon')
plt.title('Final Validation Loss Comparison')
plt.xlabel('Model')
plt.ylabel('Validation Loss')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()